In [2]:
import pandas as pd
import numpy as np
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)['cast'].values

array(['[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "ge

In [5]:
movies = movies.merge(credits, on='title')

In [6]:
movies = movies[['title', 'movie_id', 'genres', 'overview', 'cast', 'crew', 'keywords']]
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     4809 non-null   object
 1   movie_id  4809 non-null   int64 
 2   genres    4809 non-null   object
 3   overview  4806 non-null   object
 4   cast      4809 non-null   object
 5   crew      4809 non-null   object
 6   keywords  4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [7]:
movies.dropna(inplace=True)
movies.isnull().sum()
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     4806 non-null   object
 1   movie_id  4806 non-null   int64 
 2   genres    4806 non-null   object
 3   overview  4806 non-null   object
 4   cast      4806 non-null   object
 5   crew      4806 non-null   object
 6   keywords  4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.4+ KB


In [8]:
movies.duplicated().sum()
movies['genres'].head(1)

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [9]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [10]:
def convert2(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
movies['cast'] = movies['cast'].apply(convert2)

In [11]:
movies.head(1)

,title,movie_id,genres,overview,cast,crew,keywords
0,Avatar,19995,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[culture clash, future, space war, space colon..."


In [12]:
def fetch_dir(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L
movies['crew']= movies['crew'].apply(fetch_dir)

In [13]:
movies.head(1)

,title,movie_id,genres,overview,cast,crew,keywords
0,Avatar,19995,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon..."


In [14]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [15]:
movies['tags'] = movies['genres'] + movies['overview'] + movies['cast'] + movies['crew']
movies['tags'].head()

0    [Action, Adventure, Fantasy, ScienceFiction, I...
1    [Adventure, Fantasy, Action, Captain, Barbossa...
2    [Action, Adventure, Crime, A, cryptic, message...
3    [Action, Crime, Drama, Thriller, Following, th...
4    [Action, Adventure, ScienceFiction, John, Cart...
Name: tags, dtype: object

In [16]:
new_df = movies[['title', 'tags', 'movie_id']]
new_df.shape

(4806, 3)

In [17]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\saadz\AppData\Local\Temp\ipykernel_7732\2448108189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
C:\Users\saadz\AppData\Local\Temp\ipykernel_7732\2448108189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(txt):
    y = []
    for j in txt.split():
        y.append(ps.stem(j))
    return " ".join(y)

new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\saadz\AppData\Local\Temp\ipykernel_7732\1488507154.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [22]:
new_df[new_df['title'] == 'Frozen'].index[0]

124

In [23]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)),reverse=True, key = lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [24]:
recommend('Spider-Man')

Spider-Man 3
Spider-Man 2
The Amazing Spider-Man 2
21 Jump Street
Kick-Ass


In [25]:
import pickle 
pickle.dump(new_df.to_dict(),open('movies_dict.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))
